In [1]:
import numpy as np
from scipy import io

In [2]:
img_list=[]
image_names = io.loadmat('images.mat')['images']
image_attributes = io.loadmat('attributeLabels_continuous.mat')['labels_cv']
for i in range(image_names.shape[0]):
    img_list.append(image_names[i][0][0])
att_dict = dict(zip(img_list, image_attributes))

In [3]:
res101 = io.loadmat('../../resnet-feats/SUN/res101.mat')
att_splits = io.loadmat('att_splits.mat')
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'
train_images = res101['image_files'][np.squeeze(att_splits[train_loc]-1)]
val_images = res101['image_files'][np.squeeze(att_splits[val_loc]-1)]
test_images = res101['image_files'][np.squeeze(att_splits[test_loc]-1)]
class_labels = res101['labels']
prior_matrix = att_splits['att']
test_classes = class_labels[np.squeeze(att_splits[test_loc]-1)]
prior_matrix_ts = prior_matrix[:,(np.unique(test_classes)-1)]

In [4]:
print 'Train images = ' + str(len(train_images)) + '\nVal images = ' + str(len(val_images)) + '\nTest images = ' + str(len(test_images))

Train images = 11600
Val images = 1300
Test images = 1440


In [5]:
train_att=np.zeros([train_images.shape[0], image_attributes.shape[1]])
for i in range(train_images.shape[0]):
    img_name=train_images[i][0][0].split('images/')[1]
    train_att[i] = np.round(att_dict[img_name])
val_att=np.zeros([val_images.shape[0], image_attributes.shape[1]])
for i in range(val_images.shape[0]):
    img_name=val_images[i][0][0].split('images/')[1]
    val_att[i] = np.round(att_dict[img_name])
test_att=np.zeros([test_images.shape[0], image_attributes.shape[1]])
for i in range(test_images.shape[0]):
    img_name=test_images[i][0][0].split('images/')[1]
    test_att[i] = np.round(att_dict[img_name])

In [6]:
corr_train = np.corrcoef(train_att.transpose())
nans = np.isnan(corr_train)
corr_train[nans] = 0
corr_test = np.corrcoef(prior_matrix_ts)
nans = np.isnan(corr_test)
corr_test[nans] = 0
def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)
dis_corr = diff_corr(corr_train, corr_test)

In [7]:
loc = np.unravel_index(np.argsort(-dis_corr, axis=None)[:100], dis_corr.shape)
dis_corr[loc]

array([0.55039062, 0.55039062, 0.41646005, 0.41646005, 0.41448207,
       0.41448207, 0.40933447, 0.40933447, 0.3882846 , 0.3882846 ,
       0.38106693, 0.38106693, 0.36520544, 0.36520544, 0.35411016,
       0.35411016, 0.35328142, 0.35328142, 0.3480737 , 0.3480737 ,
       0.34444056, 0.34444056, 0.33448378, 0.33448378, 0.33150595,
       0.33150595, 0.33143327, 0.33143327, 0.31892079, 0.31892079,
       0.3174054 , 0.3174054 , 0.31633169, 0.31633169, 0.31623479,
       0.31623479, 0.30111212, 0.30111212, 0.29919132, 0.29919132,
       0.29809287, 0.29809287, 0.29499453, 0.29499453, 0.29404743,
       0.29404743, 0.287901  , 0.287901  , 0.28564063, 0.28564063,
       0.28428015, 0.28428015, 0.28398512, 0.28398512, 0.28386863,
       0.28386863, 0.27735522, 0.27735522, 0.27047488, 0.27047488,
       0.26675528, 0.26675528, 0.26634216, 0.26634216, 0.26162947,
       0.26162947, 0.26094019, 0.26094019, 0.26090891, 0.26090891,
       0.25988597, 0.25988597, 0.25717829, 0.25717829, 0.25517

In [8]:
dis_corr.mean()

0.015219056458505086

<br>
<br>
<br>
### New split for SUN dataset

In [10]:
res101['image_files']

array([[array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/a/abbey/sun_aakbdcgfpksytcwj.jpg'],
      dtype='<U76')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/a/abbey/sun_aaoktempcmudsvna.jpg'],
      dtype='<U76')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/a/abbey/sun_abegcweqnetpdlrh.jpg'],
      dtype='<U76')],
       ...,
       [array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/z/zoo/sun_bvyaxgkxlaeedguu.jpg'],
      dtype='<U74')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/z/zoo/sun_bxjkxzvljdwtnjud.jpg'],
      dtype='<U74')],
       [array([u'/BS/Deep_Fragments/work/MSc/data/SUN/images/z/zoo/sun_bxydkwwiknutjnun.jpg'],
      dtype='<U74')]], dtype=object)

In [17]:
all_img = att_dict.keys()
cls = [img.split('/')[1] for img in all_img]
len(np.unique(cls))

611

In [19]:
res101['labels']

array([[  1],
       [  1],
       [  1],
       ...,
       [717],
       [717],
       [717]], dtype=uint16)

In [23]:
att_splits = io.loadmat('att_splits.mat') #ZSL_GBU data
image_names = np.array([x[0][0].split('images/')[1] for x in res101['image_files']])
class_labels = res101['labels']
name2class = dict(zip(image_names, np.squeeze(class_labels)))

In [28]:
class2name = {}
for n in name2class:
    c = name2class[n]
    if c not in class2name:
        class2name[c] = [n]
    else:
        class2name[c].append(n)

In [30]:
def diff_corr(corr_train, corr_test):
    dis_corr = (corr_train - corr_test)
    dis_corr = np.sign(corr_train)*dis_corr
    return dis_corr.clip(0,np.inf)

In [84]:
def get_corr_shift(att_dict, class2name, train_class, test_class):
    test_att_mat = []
    train_att_mat = []
    for cls in train_class:
        for img in class2name[cls]:
            train_att_mat.append(att_dict[img])
    for cls in test_class:
        for img in class2name[cls]:
            test_att_mat.append(att_dict[img])
#     print '#Train instances = ' + str(len(train_att_mat)) + '\n#Test instances = ' + str(len(test_att_mat))
    
    train_att_mat = np.array(train_att_mat).transpose()
    test_att_mat = np.array(test_att_mat).transpose()
    
    corr_train = np.corrcoef(train_att_mat)
    corr_train[np.isnan(corr_train)] = 0.
    corr_test = np.corrcoef(test_att_mat)
    corr_test[np.isnan(corr_test)] = 0.
    
    dis_corr = diff_corr(corr_train, corr_test)
    
    # correlation shift score: 
    # 1) mean
    # corr_shift_score = np.mean(dis_corr)
    
    # 2) average of top n%
    dis_corr_array = dis_corr.flatten()
    top_percentage = 100
    num_elements = int((top_percentage/100.)*len(dis_corr_array))
    corr_shift_score = np.mean(dis_corr_array[np.argsort(dis_corr_array)[-num_elements:]])
    
    return corr_shift_score

In [55]:
train_classes = np.unique([name2class['/'.join(tr[0][0].split('/')[8:])] for tr in train_images])
val_classes = np.unique([name2class['/'.join(val[0][0].split('/')[8:])] for val in val_images])
test_classes = np.unique([name2class['/'.join(te[0][0].split('/')[8:])] for te in test_images])

In [77]:
get_corr_shift(att_dict, class2name, train_classes, test_classes)

0.03247590150450682

In [74]:
import time

selected_classes = []
remaining_classes = range(718)
remaining_classes.remove(0)

start_time = time.time()
for i in range(72):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(att_dict, class2name, new_rem, new_sel)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + str(selected_classes[-1]) + ' -> ' + str(best_score) + ' :' + str(time.time() - start_time)+ 's' 

test_set = selected_classes[:]
train_val = remaining_classes[:]

1) 20 -> 0.20462419926437994 :18.3212909698s
2) 141 -> 0.2093474935863399 :36.9060668945s
3) 583 -> 0.20047480836126033 :56.3604898453s
4) 662 -> 0.19322437318073618 :73.2223098278s
5) 663 -> 0.18833242156442223 :92.0463619232s
6) 660 -> 0.1837130609114906 :110.226444006s
7) 388 -> 0.1802782199504697 :129.345793009s
8) 664 -> 0.17829927410810625 :149.48506999s
9) 661 -> 0.17622179441352848 :169.875679016s
10) 666 -> 0.17429209823472316 :188.391891003s
11) 609 -> 0.17174128692156115 :207.412191868s
12) 145 -> 0.16907402098510294 :225.539800882s
13) 156 -> 0.16604607730706417 :243.031993866s
14) 455 -> 0.1634614413357417 :261.67262888s
15) 460 -> 0.16136732161467804 :279.927438021s
16) 486 -> 0.15907406963691845 :299.908885002s
17) 94 -> 0.15716594599212874 :318.51018095s
18) 193 -> 0.15553520096515455 :335.962906837s
19) 414 -> 0.15411966550812403 :353.242909908s
20) 615 -> 0.15254917703328633 :372.411615849s
21) 535 -> 0.1519518071301455 :389.708205938s
22) 552 -> 0.15070018122676382 :

In [78]:
all_classes = range(718)
all_classes.remove(0)
train_val = all_classes[:]
for cls in all_classes:
    if cls in test_set:
        train_val.remove(cls) 
len(train_val)

645

In [81]:
selected_classes = []
remaining_classes = train_val[:]
for i in range(65):
    best_cls = ''
    best_score = -1.
    for cls in remaining_classes:
        new_sel = selected_classes[:]
        new_sel.append(cls)
        new_rem = remaining_classes[:]
        new_rem.remove(cls)
        shift_score = get_corr_shift(att_dict, class2name, new_rem, test_set)
        if shift_score > best_score:
            best_score = shift_score
            best_cls = cls
    selected_classes.append(best_cls)
    remaining_classes.remove(best_cls)
    print str(i+1) + ') ' + str(selected_classes[-1]) + ' -> ' + str(best_score)
train_set = remaining_classes[:]
val_set = selected_classes[:]

1) 354 -> 0.1269732853018949
2) 345 -> 0.12722343007394799
3) 538 -> 0.12740130037072006
4) 283 -> 0.1276036353217557
5) 271 -> 0.12783243105137734
6) 477 -> 0.12802720410429658
7) 219 -> 0.12822483603226514
8) 655 -> 0.12841089436826603
9) 82 -> 0.12859504961985427
10) 508 -> 0.1287134120009919
11) 270 -> 0.12883888644085156
12) 334 -> 0.12899817623008233
13) 333 -> 0.12913922529884012
14) 416 -> 0.12926684676178427
15) 494 -> 0.1294062882700574
16) 53 -> 0.1295847677916621
17) 261 -> 0.1297270759993281
18) 516 -> 0.12987119027626
19) 233 -> 0.13003370154504618
20) 604 -> 0.13028866986827206
21) 697 -> 0.13044779282426364
22) 253 -> 0.13059373657837528
23) 254 -> 0.13073933173013058
24) 690 -> 0.13087716503395916
25) 308 -> 0.13099731578571006
26) 296 -> 0.13111697699816074
27) 502 -> 0.13124521639087916
28) 596 -> 0.1313744891650859
29) 554 -> 0.13150024274430447
30) 120 -> 0.1316546848234275
31) 162 -> 0.1317680650012808
32) 349 -> 0.13192344779564108
33) 524 -> 0.13202762130807577


In [83]:
print get_corr_shift(att_dict, class2name, train_classes, test_classes)
print get_corr_shift(att_dict, class2name, train_set, test_set)

0.03247590150450682
0.13562266256141875


In [85]:
print get_corr_shift(att_dict, class2name, train_classes, test_classes)
print get_corr_shift(att_dict, class2name, train_set, test_set)

0.016239680487793468
0.07378322213034062


In [87]:
test_count = 0
for cls in test_set:
    test_count += len(class2name[cls])

In [88]:
test_count

1440

In [90]:
split_dict = {}
split_dict['train_cls'] = train_set
split_dict['val_cls'] = val_set
split_dict['test_cls'] = test_set

import pickle
with open('sun_cs_split.npy', 'wb') as fp:
    np.save(fp, split_dict)

In [92]:
sd = np.load('sun_cs_split.npy', allow_pickle=True).item()
sd

{'test_cls': [20,
  141,
  583,
  662,
  663,
  660,
  388,
  664,
  661,
  666,
  609,
  145,
  156,
  455,
  460,
  486,
  94,
  193,
  414,
  615,
  535,
  552,
  379,
  52,
  355,
  673,
  112,
  36,
  674,
  438,
  318,
  119,
  307,
  154,
  258,
  348,
  614,
  665,
  292,
  14,
  580,
  581,
  347,
  195,
  520,
  415,
  585,
  476,
  23,
  362,
  363,
  582,
  579,
  227,
  163,
  559,
  505,
  179,
  532,
  572,
  548,
  246,
  152,
  702,
  454,
  599,
  401,
  380,
  330,
  108,
  249,
  443],
 'train_cls': [1,
  2,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  15,
  16,
  18,
  19,
  21,
  22,
  24,
  25,
  26,
  27,
  28,
  30,
  31,
  32,
  33,
  34,
  35,
  37,
  38,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  77,
  78,
  79,
  80,
  81,
  83,
  84,
  85,
  86,
  88,
  89,
  90,
  91,
  92,
  93,
  95,
  96,
  97

In [99]:
min(train_set+val_set+test_set)

1

In [100]:
train_loc = []
val_loc = []
test_loc = []

for i, label in enumerate(class_labels):
    if label in sd['train_cls']:
        train_loc.append(i)
    elif label in sd['val_cls']:
        val_loc.append(i)
    elif label in sd['test_cls']:
        test_loc.append(i)